<a href="https://colab.research.google.com/github/apkendall10/NBAValueFinder/blob/master/Linear_NBA_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import and set up initial conditions

from __future__ import print_function
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


In [0]:
def remove_dups(data,year):
  uniqPlayer=pd.DataFrame()
  temp_data=data.copy()
  #temp_data.set_index("Rk",inplace=True)
  #print(temp_data.index)
  firstTime=0
  for rk in temp_data.index:
    #print(rk)
    player=temp_data.loc[rk,'Player']
    #print(player)
    player=player[player.find("\\")+1:player.find("\\")+10]
    temp_data.loc[rk,'PlayerID']=player
    player = player + str(year)
    temp_data.loc[rk,'PlayerYID']= player
    temp_data.loc[rk,'year']= year
    temp_data.loc[rk,"FPA"]=( temp_data.loc[rk,"PTS"]+ temp_data.loc[rk,"AST"]+ temp_data.loc[rk,"STL"]+ temp_data.loc[rk,"TRB"]
      + temp_data.loc[rk,"BLK"]- temp_data.loc[rk,"TOV"]- temp_data.loc[rk,"FGA"]- temp_data.loc[rk,"FTA"]+ temp_data.loc[rk,"FG"]+ temp_data.loc[rk,"FT"])/ temp_data.loc[rk,'G']
    if player not in uniqPlayer.index:
      if temp_data.loc[rk,'G']>10 and temp_data.loc[rk,'PTS']>50:
        addData=temp_data.loc[rk:rk]
        add=pd.DataFrame(addData, columns=temp_data.columns)
        if firstTime==0:
          uniqPlayer=add
          uniqPlayer.set_index('PlayerYID',inplace=True)
          firstTime=1
        else:
          add.set_index('PlayerYID',inplace=True)
          uniqPlayer=uniqPlayer.append(add)
  return uniqPlayer
def get_year_data(year):
  path='/content/gdrive/My Drive/Sports Data/' + str(year) + '-' + str(year+1) + ' Season.txt'
  return remove_dups(pd.read_csv(path),year)
def adjust_lockout_year(data):
  for col in data.columns:
    if(col not in ['Rk','Player','Pos','Age','Tm','FG%','3P%','2P%','eFG%','FT%','PlayerID','year','FPA']):
      data[col]=data[col]*82/66
  return data

In [0]:
data = get_year_data(2007)
for year in range(2008,2016):
  new_year=get_year_data(year)
  if(year==2011):
    new_year=adjust_lockout_year(new_year)
  data = data.append(new_year)
data_sub = get_year_data(2017)
data2_sub = get_year_data(2018)
data2 = get_year_data(2017)

In [0]:
def preprocess_year_link(init_data,sub_data):
  data=init_data.copy()
  data=data.append(sub_data)
  init_data['next year fantasy points']=0
  for player in init_data.index:
    #print(player)
    pid=data.loc[player,'PlayerID']
    next_yid=pid + str(int(data.loc[player,'year']+1)) #remove +1 for testing
    #print(next_yid)
    if next_yid in data.index:
      #print(next_yid)
      init_data.loc[player,'next year fantasy points']=data.loc[next_yid,"FPA"]
      #print(data.loc[next_yid,"FPA"])
    else:
      init_data=init_data.drop(player, axis=0)
      #print(player)
  return init_data
data = preprocess_year_link(data,data_sub)
data2 = preprocess_year_link(data2,data2_sub)

In [0]:
def preprocess_features(data):
  selected_features = data[
    ["Age",
    "G",
     "MP",
     "GS",
     "FG",
     "FGA",
     #"FG%",
     "3P",
     "3PA",
     #"3P%",
     "2P",
     "2PA",
     "2P%",
     #"eFG%",
     "FT",
     "FTA",
     #"FT%",
     "ORB",
     "DRB",
     "TRB",
     "AST",
     "STL",
     "BLK",
     "TOV",
     "PF",
     "PTS",
     "FPA"
    ]]
  processed_features = selected_features.copy()
  return processed_features

def preprocess_targets(init_data,sub_data):
  output_targets = pd.DataFrame()
  output_targets["fantasy points"] = init_data['next year fantasy points']
  return output_targets

In [0]:
# Create training and validation info.
training_examples = preprocess_features(data)
training_targets = preprocess_targets(data,data_sub)
validation_examples = preprocess_features(data2)
validation_targets = preprocess_targets(data2,data2_sub)

In [0]:
#training_targets.sort_values(by='fantasy points')
#training_examples.describe()

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                            
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
def get_quantile_based_boundaries(feature_values, num_buckets):
  boundaries = np.arange(1.0, num_buckets) / num_buckets
  quantiles = feature_values.quantile(boundaries)
  return [quantiles[q] for q in quantiles.keys()]
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
    #"Age",    "G",     "MP",    "GS",     "FG",     "FGA",     "FG%",     "3P",
     "3PA",     "3P%",     "2P",     "2PA",     "2P%",     "eFG%",     "FT",     "FTA",     "FT%",     "ORB",
     "DRB",     "TRB",     "AST",     "STL",     "BLK",     "TOV",     "PF",     "PTS", "FPA"
  """ 
  fpa = tf.feature_column.numeric_column("FPA")
  ast = tf.feature_column.numeric_column("AST")
  stl = tf.feature_column.numeric_column("STL")
  blk = tf.feature_column.numeric_column("BLK")
  tov = tf.feature_column.numeric_column("TOV")

# Divide age into 5 buckets.
  age = tf.feature_column.numeric_column("Age")
  bucketized_age = tf.feature_column.bucketized_column(
    age, boundaries=get_quantile_based_boundaries(
      input_features["Age"], 5))
  
  return set([fpa, ast, stl, blk, tov, bucketized_age])

In [0]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a neural network regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.
      
  Returns:
    A `DNNRegressor` object trained on the training data.
  """

  periods = 10
  steps_per_period = steps / periods
  
  # Create a LinearRegressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer,
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["fantasy points"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["fantasy points"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["fantasy points"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  print("Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error)
  print("Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error)

  return linear_regressor

In [0]:
linear_regressor = train_model(
    learning_rate=0.001,
    steps=50000,
    batch_size=150,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

In [0]:
data_2018=get_year_data(2018)
data_2018['next year fantasy points']=data_2018['FPA']
predict_testing_input_fn = lambda: my_input_fn(preprocess_features(data_2018), 
                                               preprocess_targets(data_2018,data_2018)["fantasy points"], 
                                               num_epochs=1, 
                                               shuffle=False)

test_predictions = linear_regressor.predict(input_fn=predict_testing_input_fn)
test_predictions = np.array([item['predictions'][0]*82 for item in test_predictions])
prediction=pd.DataFrame(test_predictions, index=data_2018.index, columns=['fantasy points'])
for player in prediction.index:
  prediction.loc[player,'pos']=data_2018.loc[player,'Pos']
  name=data_2018.loc[player,'Player']
  name=name[0:(name.find("\\"))]
  prediction.loc[player,'Name']=name
prediction.to_csv('/content/gdrive/My Drive/Sports Data/prediction_age.csv')

In [0]:
data.to_csv('/content/gdrive/My Drive/Sports Data/data.csv')
training_examples.to_csv('/content/gdrive/My Drive/Sports Data/examples.csv')
training_targets.to_csv('/content/gdrive/My Drive/Sports Data/targets.csv')

In [0]:
print([linear_regressor.get_variable_value(my_var) for my_var in linear_regressor.get_variable_names()][1:])
linear_regressor.get_variable_names()